In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from te_arcolanche import *

import csv

from multiprocess import Pool,cpu_count
from itertools import product


map2_config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '92kf1c',
     'type': 'arc',
     'config': {'dataId': '256,320',
      'label': 'new layer',
      'color': [218, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'lat1',
       'lng0': 'long1',
       'lat1': 'lat2',
       'lng1': 'long2'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 2,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0.1, 2],
       'targetColor': None},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': {'name': 'TE', 'type': 'real'},
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'256,320': [{'name': 'long1',
        'format': None},
       {'name': 'lat1', 'format': None},
       {'name': 'long2', 'format': None},
       {'name': 'lat2', 'format': None},
       {'name': 'TE', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 1.7172966072248994,
   'longitude': 18.101964233149975,
   'pitch': 0,
   'zoom': 2.2866482635409926,
   'isSplit': False},
  'mapStyle': {'styleType': 'light',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': False,
    'road': False,
    'border': True,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

## Generate causal structure for a given dx and time

In [12]:
time = 64
dx = 160

dtdx = (time, dx)
gridix = 7

# load polygons
polygons = gpd.read_file(f'voronoi_grids/{dtdx[1]}/borders{str(gridix).zfill(2)}.shp')
def neighbors_to_list(neighbor_list):
    return list(map(int , neighbor_list.replace(' ', '').split(',')))
neighbor_info_df = polygons.drop('geometry' , axis=1)
neighbor_info_df['neighbors'] = neighbor_info_df['neighbors'].apply(neighbors_to_list)

df = pd.read_csv(f'generated_data/battles/gridix_{gridix}/FG_time_series/time_series_1_{dtdx[1]}.csv')
time_series = CG_time_series_fast(df.values, dtdx[0])
time_series = pd.DataFrame(time_series, columns=df.columns.astype(int) , index=range(1,len(time_series)+1))

pair_poly_te, filtered_neighbors, clean_pair_poly_te = te_causal_network(time_series, neighbor_info_df)

In [13]:
clean_pair_poly_te

[(5, 24, nan),
 (12, 14, nan),
 (12, 22, nan),
 (12, 24, nan),
 (12, 29, nan),
 (14, 12, nan),
 (14, 24, 0.02108790888355599),
 (14, 25, nan),
 (14, 29, nan),
 (14, 33, nan),
 (22, 12, nan),
 (22, 29, 0.0583787455229996),
 (22, 37, 0.08140160965447078),
 (22, 49, nan),
 (24, 5, nan),
 (24, 12, nan),
 (24, 14, nan),
 (25, 14, nan),
 (25, 33, nan),
 (25, 56, 0.0359865028150325),
 (25, 63, nan),
 (25, 71, nan),
 (29, 12, nan),
 (29, 14, nan),
 (29, 22, nan),
 (29, 33, nan),
 (29, 49, nan),
 (33, 14, nan),
 (33, 25, nan),
 (33, 29, nan),
 (33, 49, nan),
 (33, 50, nan),
 (33, 55, nan),
 (33, 71, nan),
 (37, 22, nan),
 (37, 39, nan),
 (37, 49, nan),
 (37, 104, 0.046230128638872395),
 (37, 106, nan),
 (39, 37, nan),
 (39, 104, 0.03164386813313028),
 (39, 106, nan),
 (39, 118, nan),
 (44, 80, nan),
 (44, 97, nan),
 (49, 22, 0.03857553940591093),
 (49, 29, nan),
 (49, 33, nan),
 (49, 37, nan),
 (49, 55, nan),
 (49, 106, nan),
 (50, 33, nan),
 (50, 52, nan),
 (50, 64, 0.023912775393539254),
 (50

## Visualize the causal network on African map

In [14]:
tile_pairs = np.array(clean_pair_poly_te)
tile_pairs = tile_pairs[np.isnan(tile_pairs[:,2]) != True]

pol1 = polygons.loc[tile_pairs[:,0]]["geometry"]
pol2 = polygons.loc[tile_pairs[:,1]]["geometry"]

centroid1 = pol1.centroid.reset_index(drop=True)
centroid2 = pol2.centroid.reset_index(drop=True)

TE_links_data = pd.DataFrame(columns=["one_lng" , "one_lat" , "two_lng" , "two_lat" , "TE"])

one_lng = [list(centroid1[i].coords)[0][0] for i in range(len(centroid1))]
one_lat = [list(centroid1[i].coords)[0][1] for i in range(len(centroid1))]

two_lng = [list(centroid2[i].coords)[0][0] for i in range(len(centroid2))]
two_lat = [list(centroid2[i].coords)[0][1] for i in range(len(centroid2))]

TE_links_data["one_lng"] = one_lng
TE_links_data["one_lat"] = one_lat
TE_links_data["two_lng"] = two_lng
TE_links_data["two_lat"] = two_lat

TE_links_data["TE"] = tile_pairs[:,2]

<ipython-input-14-71adb05e7df4>:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid1 = pol1.centroid.reset_index(drop=True)
<ipython-input-14-71adb05e7df4>:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid2 = pol2.centroid.reset_index(drop=True)


In [15]:
from keplergl import KeplerGl
map2=KeplerGl(height=800)

map2.config = map2_config

map2.add_data(data=TE_links_data, name=f"{str(time)},{str(dx)}")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [16]:
map2

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '92kf1c', 'type': '…